In [2]:
from keras.layers import Conv2D, SeparableConv2D, Input
from keras.models import Model

In [3]:
input_image = Input((224, 224, 3))
feature_maps = Conv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = Conv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


###### 可以看到經過兩次 Conv2D，如果沒有設定 padding="SAME"，圖就會越來越小，同時特徵圖的 channel 數與 filters 的數量一致

In [5]:
input_image = Input((224, 224, 3))
feature_maps = SeparableConv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = SeparableConv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

In [6]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 222, 222, 32)      155       
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 220, 220, 64)      2400      
Total params: 2,555
Trainable params: 2,555
Non-trainable params: 0
_________________________________________________________________


#### 作業
請閱讀 Keras 官方範例 mnist_cnn.py

並回答下列問題。僅有 70 行程式碼，請確保每一行的程式碼你都能夠理解目的

1.是否有對資料做標準化 (normalization)? 如果有，在哪幾行?
2.使用的優化器 Optimizer 為何?
3.模型總共疊了幾層卷積層?
4.模型的參數量是多少?

In [9]:
# Introduction
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''
# import packages, mnist dataset and keras
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# set parameter as follow
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions 28*28
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 判斷圖片格式是否為channels_first，如果是就設定input_shape的第一維是channel，否則放在最後一維
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
# set the format of pixel as float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalization
x_train /= 255
x_test /= 255

# print out the shape of training data & testing data
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#build CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_______________________________________________

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
#train model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
#evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 139s 2ms/step - loss: 0.2559 - accuracy: 0.9221 - val_loss: 0.0540 - val_accuracy: 0.9826
Epoch 2/12
60000/60000 [==============================] - 131s 2ms/step - loss: 0.0871 - accuracy: 0.9740 - val_loss: 0.0428 - val_accuracy: 0.9856
Epoch 3/12
60000/60000 [==============================] - 131s 2ms/step - loss: 0.0654 - accuracy: 0.9806 - val_loss: 0.0405 - val_accuracy: 0.9865
Epoch 4/12
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0548 - accuracy: 0.9835 - val_loss: 0.0345 - val_accuracy: 0.9879
Epoch 5/12
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0465 - accuracy: 0.9858 - val_loss: 0.0321 - val_accuracy: 0.9896
Epoch 6/12
60000/60000 [==============================] - 131s 2ms/step - loss: 0.0409 - accuracy: 0.9876 - val_loss: 0.0281 - val_accuracy: 0.9905
Epoch 7/12
600

###### Answers:
###### 1.是否有對資料做標準化 (normalization)? 如果有，在哪幾行? -> 有，在第37、38行有對像素除以255進行標準化
###### 2.使用的優化器 Optimizer 為何? -> 在程式第60行使用Adadelta優化器
###### 3.模型總共疊了幾層卷積層? -> 2層，分別在程式第48及第50行
###### 4.模型的參數量是多少? -> 1199882